<a href="https://colab.research.google.com/github/Neilus03/NLP-2023/blob/main/exercise3_real_word_spellchecker_neil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import os

In [ ]:
words = None
if False:
    from nltk.corpus import gutenberg as corpus
    words = corpus.words()
    vocabulary = vocabulary[803:-2]
    # first 803 and last 2 words are punctuation signs, numbers and underscored 
    # words like _home_
else:
    # Using the whole corpus means too much work to later compute the words at a 
    # distance for each word in the vocabulary, so we load just one book
    import nltk
    nltk.download('gutenberg')
    words = nltk.corpus.gutenberg.words('austen-emma.txt')
    # using all the words in the book will take 25-30 min. to process later
    # so we limit its number for the moment.
    words = words[:10000] # cell [15] will take 1.5 min. 
    #words = words[:100000] # cell [15] will take 15 min. 


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
vocabulary = list(set(words))
vocabulary.sort()
# get rid of some non-words like ',' '--' '['
idx_first_word = vocabulary.index('A')
vocabulary = vocabulary[idx_first_word:]
# plus some more annoying non-words
vocabulary.remove('[')
vocabulary.remove(']')
vocabulary.remove('`')
vocabulary.remove('II')
vocabulary = np.array(vocabulary)
print(vocabulary)
print('Vocabulary size {} words'.format(len(vocabulary)))

['A' 'Abbey' 'After' ... 'yourself' 'youth' 'youthful']
Vocabulary size 1711 words


For each word in the vocabulary find the nearest words = at Levenshtein distance up to ``MAX_DIST``. This is a long computation, $O(n^2)$ for $n$ size of the vocabulary. We try to speed up it a little : if $\text{dist}(w_1, w_2) \leq d$ then $|\text{len}(w_1) - \text{len}(w_2)| \leq d$. This reduces the candidate words in the vocabulary for which to compute the distance to each word.
We save the resulting dictionary to avoid recomputing it each time.

In [ ]:
from nltk.metrics.distance import edit_distance

def levenshtein(s1, s2):
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

word_lengths = np.array([len(w) for w in vocabulary])
dict_lengths = {}
for l in range(min(word_lengths), max(word_lengths)+1):
    dict_lengths[l] = vocabulary[word_lengths==l] # needs vocabulary to be a numpy array

min_length = min(dict_lengths)
max_length = max(dict_lengths)

MAX_DIST = 1
fname = 'close_words_{}.pkl'.format(MAX_DIST)
if not os.path.exists(fname):
    close_words = {}
    for word in tqdm(vocabulary):
        length = len(word)
        candidate_words = []
        d1 = max(min_length, length - MAX_DIST)
        d2 = min(max_length, length + MAX_DIST)
        for d in range(d1, d2+1):
            candidate_words.extend(dict_lengths[d])
        close_words[word] = [w for w in candidate_words if levenshtein(word,w) <= MAX_DIST]
    #becareful with pickle, remove the file so you can recompute the distances for new words in next batch
    with open(fname,'wb') as f:
        pickle.dump(close_words, f)
else:
    close_words = pickle.load(open(fname,'rb'))

print(close_words)

{'A': ['A', 'I', 'a', 's', 'Ah', 'An', 'As', 'At'], 'Abbey': ['Abbey'], 'After': ['After', 'after'], 'Agricultural': ['Agricultural'], 'Ah': ['A', 'Ah', 'An', 'As', 'At', 'Oh'], 'Alderneys': ['Alderneys'], 'All': ['All', 'all', 'ill'], 'Altogether': ['Altogether', 'altogether'], 'An': ['A', 'Ah', 'An', 'As', 'At', 'In', 'an', 'in', 'on', 'And'], 'And': ['An', 'And', 'and', 'end'], 'As': ['A', 's', 'Ah', 'An', 'As', 'At', 'as', 'is', 'us'], 'At': ['A', 'Ah', 'An', 'As', 'At', 'It', 'at', 'it'], 'Austen': ['Austen'], 'Bates': ['Bates'], 'Being': ['Being', 'being'], 'Between': ['Between', 'between'], 'Boarding': ['Boarding'], 'Broadway': ['Broadway'], 'Brunswick': ['Brunswick'], 'But': ['But', 'but', 'out', 'put'], 'By': ['By', 'My', 'by', 'my'], 'CHAPTER': ['CHAPTER'], 'Captain': ['Captain'], 'Children': ['Children', 'children'], 'Christmas': ['Christmas'], 'Churchill': ['Churchill', 'Churchills'], 'Churchills': ['Churchill', 'Churchills'], 'Dear': ['Dear', 'bear', 'dear', 'hear', 'tear'

Given one sentence $X$, which is a list of words, build the candidates to correct sentence $C(X)$ assuming at most one word is mispelled.

In [ ]:
#sentence = 'Only two of the apple'
sentence = 'I wish you where here'
X = sentence.split(' ')
for x in X:
  assert x in vocabulary, 'All the words in the sentence must belong to the '\
      + 'vocabulary, {} doesn\'t'.format(x)
CX = [X] # no errors
#print(CX)
for i in range(len(X)): # one mispelled word at a time
    
    if X[i] in vocabulary: 
        for cw in close_words[X[i]]:
            if cw != X[i]:
                C = X.copy()
                C[i] = cw
                CX.append(C)
    else:
        pass # let it be as it is
print(f'Sentence that contain a mispelled word to be corrected:\n{CX[0]}\n\n\nCandidates to be the corrected sentence:\n')
for W in CX[1:]: #as I dont want to print the same sentence that was mispelled.
    print('\t'.join(W))

Sentence that contain a mispelled word to be corrected:
['I', 'wish', 'you', 'where', 'here']


Candidates to be the corrected sentence:

A	wish	you	where	here
a	wish	you	where	here
s	wish	you	where	here
IV	wish	you	where	here
If	wish	you	where	here
In	wish	you	where	here
It	wish	you	where	here
I	fish	you	where	here
I	with	you	where	here
I	wish	You	where	here
I	wish	your	where	here
I	wish	you	here	here
I	wish	you	were	here
I	wish	you	There	here
I	wish	you	Where	here
I	wish	you	there	here
I	wish	you	where	her
I	wish	you	where	hers
I	wish	you	where	were
I	wish	you	where	There
I	wish	you	where	Where
I	wish	you	where	there
I	wish	you	where	where


Likelihood $P(X | W) = \prod_{i=1}^n p(x_i | w_i)$ where $n$ is number of words in $X$ (same as in $W$), and $p(x | w)$  is Eq. B.8. $X$ is the written sentence, $W$ are the candidate sentences in $C(X)$. Each $W$ contains zero (ie, $W=X$) or at most one mispelled word, and in this case the mispelled word $w_i$ is at a Levenshtein distance $1...$ ``MAX_DIST`` of the written word $x_i$ 

In [ ]:
alpha = 0.95
likelihoods = []
for W in CX:
    PXW = 1.0
    #print(X,W)
    for x,w in zip(X,W):
        if w==x:
            pxw = alpha
        else:
            close_to_x = close_words[x] # includes x itself
            pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
        PXW *= pxw
    likelihoods.append(PXW)

likelihoods = np.array(likelihoods)
idx_most_likely = likelihoods.argmax()
print('Sentence with highest likelihood is the written one, X:')
print('\t'.join(CX[idx_most_likely]), '\t', likelihoods[idx_most_likely])
print('\n\nLikelihoods\n')
num_candidates = len(CX)
for i in range(1,num_candidates): #the 1 is not to print again the likelihood of X as wehave just printed before
    print('\t'.join(CX[i]), '\t', likelihoods[i])


Sentence with highest likelihood is the written one, X:
I	wish	you	where	here 	 0.7737809374999999


Likelihoods

A	wish	you	where	here 	 0.005817901785714291
a	wish	you	where	here 	 0.005817901785714291
s	wish	you	where	here 	 0.005817901785714291
IV	wish	you	where	here 	 0.005817901785714291
If	wish	you	where	here 	 0.005817901785714291
In	wish	you	where	here 	 0.005817901785714291
It	wish	you	where	here 	 0.005817901785714291
I	fish	you	where	here 	 0.020362656250000017
I	with	you	where	here 	 0.020362656250000017
I	wish	You	where	here 	 0.020362656250000017
I	wish	your	where	here 	 0.020362656250000017
I	wish	you	here	here 	 0.008145062500000006
I	wish	you	were	here 	 0.008145062500000006
I	wish	you	There	here 	 0.008145062500000006
I	wish	you	Where	here 	 0.008145062500000006
I	wish	you	there	here 	 0.008145062500000006
I	wish	you	where	her 	 0.00581790178571429
I	wish	you	where	hers 	 0.00581790178571429
I	wish	you	where	were 	 0.00581790178571429
I	wish	you	where	There 	 0.00581

Priors $P(W)$ for all $W \in C(X)$ computed by a LM model, for instance tri-grams (on this same corpus). We've chosen the stupid backoff version.

In [ ]:
nltk.download('punkt')

sents = nltk.corpus.gutenberg.sents('austen-emma.txt')
text = []
for s in tqdm(sents):
    text.append(s[:-1]) # except ending point

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import StupidBackoff

n=3
lm = StupidBackoff(alpha=0.4, order=n)
train, vocab = padded_everygram_pipeline(n, text)
lm.fit(train, vocab)
print(lm.counts)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 7752/7752 [00:00<00:00, 19082.32it/s]


<NgramCounter with 3 ngram orders and 623964 ngrams>


In [ ]:
def P(W, verbose=False):
    S = ['<s>', '<s>',] + W + ['</s>'] 
    if verbose: print(S)
    num_words = len(S)
    PW = 1.0
    for i in range(2,num_words-1): # omit </s> because likelihoods don't have it
        score = lm.score(S[i], [S[i-2], S[i-1]])
        if verbose: print('P({} | {}, {}) = {}'.format(S[i], S[i-2], S[i-1], score))
        PW *= score
    return PW

print('X =', X)
score = P(X, verbose=True)
print('P(X) = {}'.format(score))
for i in range(len(CX)):
    print('\nW = CX[i] = {}'.format(CX[i]))
    score = P(CX[i], verbose=True)
    print('P(W) = {}'.format(score))

X = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(X) = 1.1475972685804098e-11

W = CX[i] = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.1475972685804098e-11

W = CX[i] = ['A', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'A', 'wish', 'you', 'where', 'here', '</s>']
P(A | <s>, <s>) = 0.011996904024767802
P(wish | <s>, A) = 9.937888198757765e-05
P(you | A, wish) = 0.04477611940298508
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.25596524

In [ ]:
priors = []
for W in CX:
    priors.append(P(W))
    
posteriors = np.array(priors)*np.array(likelihoods)
idx_best_post = np.argmax(posteriors)

for i in range(num_candidates):
    best = '<----' if i==idx_best_post else ''
    print('\t'.join(CX[i]), '\tlikelihood = {}\tprior = {}\tposterior = {} {}'
          .format(likelihoods[i], priors[i], posteriors[i],best))

print('\nThe original sentence was:')
print('\t' + ' '.join(X))
print('The right sentence is:')
print('\t' + ' '.join(CX[idx_best_post]))

I	wish	you	where	here 	likelihood = 0.7737809374999999	prior = 1.1475972685804098e-11	posterior = 8.879888903545887e-12 
A	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 1.2559652429973822e-15	posterior = 7.307082430029553e-18 
a	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 1.8095085058862247e-16	posterior = 1.0527542767660666e-18 
s	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 3.1250979180599283e-16	posterior = 1.818151275801287e-18 
IV	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 0.0	posterior = 0.0 
If	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 9.993701933527558e-16	posterior = 5.8142376324966335e-18 
In	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 7.292701410952542e-16	posterior = 4.242822056146192e-18 
It	wish	you	where	here 	likelihood = 0.005817901785714291	prior = 3.983975770798148e-15	posterior = 2.3178379751169013e-17 
I	fish	you	where	here 	likelihood = 0.0203626562500

That's it! Now, let's try this code and do some modifications:
1. change probabilities by log of probabilities to get scores more easy to interpret
1. relax the assumption that in a sentence there's at most 1 mispelled word, what if we suppose there may be at most 2 ?
1. and what is the effect of $\alpha$ ? change it to 0.9, 0.8...
1. make a long (>20) list of sentences each with 1 or 2 mispelled words to assess the speller : does it really work ? 
1. same for a second list of sentences withouth any spelling error (but different from any one in the selected part of the book), are there any errors ?
1. extra points: what if you change the tri-gram language model by the neural network of exercise 2 ?

### 1. change probabilities by log of probabilities to get scores more easy to interpret

In [ ]:
import math

alpha = 0.95 ################CHANGED CODE NOTED BY '#' #######################
log_likelihoods = [] ##############################
for W in CX: #######################################
    log_PXW = 0.0 #######################################
    #print(X,W)
    for x, w in zip(X, W):#######################################
        if w == x:#######################################
            pxw = alpha#######################################
        else:#######################################
            close_to_x = close_words[x]  # includes x itself#######################################
            pxw = (1 - alpha) / (len(close_to_x) - 1)  # so we subtract 1#######################################
        log_PXW += math.log(pxw)#######################################
    log_likelihoods.append(log_PXW)#######################################

log_likelihoods = np.array(log_likelihoods)
idx_most_likely = log_likelihoods.argmax()
print('Sentence with highest likelihood is the written one, X:')
print('\t'.join(CX[idx_most_likely]), '\t', log_likelihoods[idx_most_likely])
print('\n\nLikelihoods\n')
num_candidates = len(CX)
for i in range(1,num_candidates): #the 1 is not to print again the likelihood of X as wehave just printed before
    print('\t'.join(CX[i]), '\t', likelihoods[i])


Sentence with highest likelihood is the written one, X:
I	wish	you	where	here 	 -0.2564664719377529


Likelihoods

A	wish	you	where	here 	 0.005817901785714291
a	wish	you	where	here 	 0.005817901785714291
s	wish	you	where	here 	 0.005817901785714291
IV	wish	you	where	here 	 0.005817901785714291
If	wish	you	where	here 	 0.005817901785714291
In	wish	you	where	here 	 0.005817901785714291
It	wish	you	where	here 	 0.005817901785714291
I	fish	you	where	here 	 0.020362656250000017
I	with	you	where	here 	 0.020362656250000017
I	wish	You	where	here 	 0.020362656250000017
I	wish	your	where	here 	 0.020362656250000017
I	wish	you	here	here 	 0.008145062500000006
I	wish	you	were	here 	 0.008145062500000006
I	wish	you	There	here 	 0.008145062500000006
I	wish	you	Where	here 	 0.008145062500000006
I	wish	you	there	here 	 0.008145062500000006
I	wish	you	where	her 	 0.00581790178571429
I	wish	you	where	hers 	 0.00581790178571429
I	wish	you	where	were 	 0.00581790178571429
I	wish	you	where	There 	 0.0058

In [ ]:
nltk.download('punkt')

sents = nltk.corpus.gutenberg.sents('austen-emma.txt')
text = []
for s in tqdm(sents):
    text.append(s[:-1])  # except ending point

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import StupidBackoff

n = 3
lm = StupidBackoff(alpha=0.4, order=n)
train, vocab = padded_everygram_pipeline(n, text)
lm.fit(train, vocab)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 7752/7752 [00:00<00:00, 11975.02it/s]


In [ ]:
def log_P(W, verbose=False):
    S = ['<s>', '<s>'] + W + ['</s>']
    if verbose: print(S)
    num_words = len(S)
    log_PW = 0.0
    epsilon = 1e-20  # small value to avoid log(0)
    for i in range(2, num_words - 1):  # omit </s> because likelihoods don't have it
        score = lm.score(S[i], [S[i - 2], S[i - 1]])
        if verbose: print('P({} | {}, {}) = {}'.format(S[i], S[i - 2], S[i - 1], score))
        log_PW += math.log(score + epsilon)
    return log_PW


print('X =', X)
score = P(X, verbose=True)
print('P(X) = {}'.format(score))
for i in range(len(CX)):
    print('\nW = CX[i] = {}'.format(CX[i]))
    score = P(CX[i], verbose=True)
    print('P(W) = {}'.format(score))

X = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(X) = 1.1475972685804098e-11

W = CX[i] = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.1475972685804098e-11

W = CX[i] = ['A', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'A', 'wish', 'you', 'where', 'here', '</s>']
P(A | <s>, <s>) = 0.011996904024767802
P(wish | <s>, A) = 9.937888198757765e-05
P(you | A, wish) = 0.04477611940298508
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.25596524

In [ ]:
log_priors = []
for W in CX:
    log_priors.append(log_P(W))

log_posteriors = np.array(log_priors) + np.array(log_likelihoods)
idx_best_post = np.argmax(log_posteriors)

for i in range(num_candidates):
    best = '<----' if i == idx_best_post else ''
    print('\t'.join(CX[i]), '\tlog_likelihood = {}\tlog_prior = {}\tlog_posterior = {} {}'
          .format(log_likelihoods[i], log_priors[i], log_posteriors[i], best))


print('\nThe original sentence was:')
print('\t' + ' '.join(X))
print('The right sentence is:')
print('\t' + ' '.join(CX[idx_best_post]))

print('\nAs it is the sentence with the highest posterior probability')

I	wish	you	where	here 	log_likelihood = -0.2564664719377529	log_prior = -25.190765597926926	log_posterior = -25.44723206986468 
A	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -34.31087200002036	log_posterior = -39.45768760017987 
a	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -36.24830622321094	log_posterior = -41.395121823370445 
s	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -35.701895871428086	log_posterior = -40.84871147158759 
IV	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -75.93946719948393	log_posterior = -81.08628279964343 
If	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -34.53940639996945	log_posterior = -39.68622200012896 
In	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -34.85448744660935	log_posterior = -40.001303046768854 
It	wish	you	where	here 	log_likelihood = -5.146815600159504	log_prior = -33.156496136833795	log_posterior = -3

### 2.relax the assumption that in a sentence there's at most 1 mispelled word, what if we suppose there may be at most 2 

In [ ]:
from nltk.metrics.distance import edit_distance

def levenshtein(s1, s2):
    return edit_distance(s1, s2, substitution_cost=1, transpositions=True)

word_lengths = np.array([len(w) for w in vocabulary])
dict_lengths = {}
for l in range(min(word_lengths), max(word_lengths)+1):
    dict_lengths[l] = vocabulary[word_lengths==l] # needs vocabulary to be a numpy array

min_length = min(dict_lengths)
max_length = max(dict_lengths)

MAX_DIST_2 = 2 ##############CHANGED VALUE###############################
fname = 'close_words_{}.pkl'.format(MAX_DIST_2)
if not os.path.exists(fname):
    close_words = {}
    for word in tqdm(vocabulary):
        length = len(word)
        candidate_words = []
        d1 = max(min_length, length - MAX_DIST_2)
        d2 = min(max_length, length + MAX_DIST_2)
        for d in range(d1, d2+1):
            candidate_words.extend(dict_lengths[d])
        close_words[word] = [w for w in candidate_words if levenshtein(word,w) <= MAX_DIST_2]
    #becareful with pickle, remove the file so you can recompute the distances for new words in next batch
    with open(fname,'wb') as f:
        pickle.dump(close_words, f)
else:
    close_words = pickle.load(open(fname,'rb'))

print(close_words)

{'A': ['A', 'I', 'a', 's', 'Ah', 'An', 'As', 'At', 'By', 'He', 'IV', 'If', 'In', 'It', 'Mr', 'My', 'No', 'Oh', 'To', 'We', 'am', 'an', 'as', 'at', 'be', 'by', 'do', 'go', 'he', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to', 'up', 'us', 'we', 'All', 'And'], 'Abbey': ['Abbey'], 'After': ['After', 'after', 'often', 'water'], 'Agricultural': ['Agricultural'], 'Ah': ['A', 'I', 'a', 's', 'Ah', 'An', 'As', 'At', 'By', 'He', 'IV', 'If', 'In', 'It', 'Mr', 'My', 'No', 'Oh', 'To', 'We', 'am', 'an', 'as', 'at', 'be', 'by', 'do', 'go', 'he', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to', 'up', 'us', 'we', 'All', 'And', 'She', 'The', 'Who', 'she', 'shy', 'the', 'who', 'why'], 'Alderneys': ['Alderneys'], 'All': ['A', 'Ah', 'An', 'As', 'At', 'All', 'And', 'all', 'ill', 'old', 'Mill', 'Well', 'call', 'fell', 'fill', 'full', 'tell', 'till', 'well', 'will'], 'Altogether': ['together', 'Altogether', 'altogether'], 'An': ['A', 'I', 'a', 's', 'Ah', 'An', 'As

In [ ]:
#sentence = 'Only two of the apples'
sentence = 'I wish you where here'
X = sentence.split(' ')
for x in X:
  assert x in vocabulary, 'All the words in the sentence must belong to the '\
      + 'vocabulary, {} doesn\'t'.format(x)
CX = [X] # no errors
#print(CX)
for i in range(len(X)): # one mispelled word at a time
    
    if X[i] in vocabulary: 
        for cw in close_words[X[i]]:
            if cw != X[i]:
                C = X.copy()
                C[i] = cw
                CX.append(C)
    else:
        pass # let it be as it is
print(f'Sentence that contain a mispelled word to be corrected:\n{CX[0]}\n\n\nCandidates to be the corrected sentence:\n')
for W in CX[1:]: #as I dont want to print the same sentence that was mispelled.
    print('\t'.join(W))

Sentence that contain a mispelled word to be corrected:
['I', 'wish', 'you', 'where', 'here']


Candidates to be the corrected sentence:

A	wish	you	where	here
a	wish	you	where	here
s	wish	you	where	here
Ah	wish	you	where	here
An	wish	you	where	here
As	wish	you	where	here
At	wish	you	where	here
By	wish	you	where	here
He	wish	you	where	here
IV	wish	you	where	here
If	wish	you	where	here
In	wish	you	where	here
It	wish	you	where	here
Mr	wish	you	where	here
My	wish	you	where	here
No	wish	you	where	here
Oh	wish	you	where	here
To	wish	you	where	here
We	wish	you	where	here
am	wish	you	where	here
an	wish	you	where	here
as	wish	you	where	here
at	wish	you	where	here
be	wish	you	where	here
by	wish	you	where	here
do	wish	you	where	here
go	wish	you	where	here
he	wish	you	where	here
if	wish	you	where	here
in	wish	you	where	here
is	wish	you	where	here
it	wish	you	where	here
me	wish	you	where	here
my	wish	you	where	here
no	wish	you	where	here
of	wish	you	where	here
on	wish	you	where	here
or	wish	you	wh

In [ ]:
alpha = 0.95
likelihoods = []
for W in CX:
    PXW = 1.0
    #print(X,W)
    for x,w in zip(X,W):
        if w==x:
            pxw = alpha
        else:
            close_to_x = close_words[x] # includes x itself
            pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
        PXW *= pxw
    likelihoods.append(PXW)

likelihoods = np.array(likelihoods)
idx_most_likely = likelihoods.argmax()
print('Sentence with highest likelihood is the written one, X:')
print('\t'.join(CX[idx_most_likely]), '\t', likelihoods[idx_most_likely])
print('\n\nLikelihoods\n')
num_candidates = len(CX)
for i in range(1,num_candidates): #the 1 is not to print again the likelihood of X as wehave just printed before
    print('\t'.join(CX[i]), '\t', likelihoods[i])

Sentence with highest likelihood is the written one, X:
I	wish	you	where	here 	 0.7737809374999999


Likelihoods

A	wish	you	where	here 	 0.0009255752840909097
a	wish	you	where	here 	 0.0009255752840909097
s	wish	you	where	here 	 0.0009255752840909097
Ah	wish	you	where	here 	 0.0009255752840909097
An	wish	you	where	here 	 0.0009255752840909097
As	wish	you	where	here 	 0.0009255752840909097
At	wish	you	where	here 	 0.0009255752840909097
By	wish	you	where	here 	 0.0009255752840909097
He	wish	you	where	here 	 0.0009255752840909097
IV	wish	you	where	here 	 0.0009255752840909097
If	wish	you	where	here 	 0.0009255752840909097
In	wish	you	where	here 	 0.0009255752840909097
It	wish	you	where	here 	 0.0009255752840909097
Mr	wish	you	where	here 	 0.0009255752840909097
My	wish	you	where	here 	 0.0009255752840909097
No	wish	you	where	here 	 0.0009255752840909097
Oh	wish	you	where	here 	 0.0009255752840909097
To	wish	you	where	here 	 0.0009255752840909097
We	wish	you	where	here 	 0.0009255752840909

In [ ]:
nltk.download('punkt')

sents = nltk.corpus.gutenberg.sents('austen-emma.txt')
text = []
for s in tqdm(sents):
    text.append(s[:-1]) # except ending point

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import StupidBackoff

n=3
lm = StupidBackoff(alpha=0.4, order=n)
train, vocab = padded_everygram_pipeline(n, text)
lm.fit(train, vocab)
print(lm.counts)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 7752/7752 [00:00<00:00, 17890.61it/s]


<NgramCounter with 3 ngram orders and 623964 ngrams>


In [ ]:
def P(W, verbose=False):
    S = ['<s>', '<s>',] + W + ['</s>'] 
    if verbose: print(S)
    num_words = len(S)
    PW = 1.0
    for i in range(2,num_words-1): # omit </s> because likelihoods don't have it
        score = lm.score(S[i], [S[i-2], S[i-1]])
        if verbose: print('P({} | {}, {}) = {}'.format(S[i], S[i-2], S[i-1], score))
        PW *= score
    return PW

print('X =', X)
score = P(X, verbose=True)
print('P(X) = {}'.format(score))
for i in range(len(CX)):
    print('\nW = CX[i] = {}'.format(CX[i]))
    score = P(CX[i], verbose=True)
    print('P(W) = {}'.format(score))

X = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(X) = 1.1475972685804098e-11

W = CX[i] = ['I', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'I', 'wish', 'you', 'where', 'here', '</s>']
P(I | <s>, <s>) = 0.08384932920536636
P(wish | <s>, I) = 0.016923076923076923
P(you | I, wish) = 0.34375
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.1475972685804098e-11

W = CX[i] = ['A', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'A', 'wish', 'you', 'where', 'here', '</s>']
P(A | <s>, <s>) = 0.011996904024767802
P(wish | <s>, A) = 9.937888198757765e-05
P(you | A, wish) = 0.04477611940298508
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 1.25596524

In [ ]:
priors = []
for W in CX:
    priors.append(P(W))
    
posteriors = np.array(priors)*np.array(likelihoods)
idx_best_post = np.argmax(posteriors)

for i in range(num_candidates):
    best = '<----' if i==idx_best_post else ''
    print('\t'.join(CX[i]), '\tlikelihood = {}\tprior = {}\tposterior = {} {}'
          .format(likelihoods[i], priors[i], posteriors[i],best))

print('\nThe original sentence was:')
print('\t' + ' '.join(X))
print('The right sentence is:')
print('\t' + ' '.join(CX[idx_best_post]))

I	wish	you	where	here 	likelihood = 0.7737809374999999	prior = 1.1475972685804098e-11	posterior = 8.879888903545887e-12 
A	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 1.2559652429973822e-15	posterior = 1.1624903865956106e-18 
a	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 1.8095085058862247e-16	posterior = 1.6748363494005602e-19 
s	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 3.1250979180599283e-16	posterior = 2.8925133933202287e-19 
Ah	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 8.103001567725047e-17	posterior = 7.499937978036197e-20 
An	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 1.215450235158757e-16	posterior = 1.1249906967054295e-19 
As	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 5.13190099289253e-16	posterior = 4.749960719422925e-19 
At	wish	you	where	here 	likelihood = 0.0009255752840909097	prior = 3.646350705476271e-16	posterior = 3.374972090116289e-19 
By	wish	

As we can see we have many more options, as we let the maximum distance be 2 instead of 1


### 3. and what is the effect of  𝛼  ? change it to 0.9, 0.8...

In [ ]:
alphas = [0.9, 0.8, 0.5, 0.1] ##########CHANGE###########
for alpha in alphas:
    alpha = alpha
    likelihoods = []
    for W in CX:
        PXW = 1.0
        #print(X,W)
        for x,w in zip(X,W):
            if w==x:
                pxw = alpha
            else:
                close_to_x = close_words[x] # includes x itself
                pxw = (1-alpha) / (len(close_to_x) - 1) # so we substract 1
            PXW *= pxw
        likelihoods.append(PXW)

    likelihoods = np.array(likelihoods)
    idx_most_likely = likelihoods.argmax()
    print('Sentence with highest likelihood is the written one, X:')
    print('\t'.join(CX[idx_most_likely]), '\t', likelihoods[idx_most_likely])
    print('\n\nLikelihoods\n')
    num_candidates = len(CX)
    for i in range(1,num_candidates): #the 1 is not to print again the likelihood of X as wehave just printed before
        print('\t'.join(CX[i]), '\t', likelihoods[i])


    print('X =', X)
    score = P(X, verbose=True)
    print('P(X) = {}'.format(score))
    for i in range(len(CX)):
        print('\nW = CX[i] = {}'.format(CX[i]))
        score = P(CX[i], verbose=True)
        print('P(W) = {}'.format(score))


    priors = []
    for W in CX:
        priors.append(P(W))
        
    posteriors = np.array(priors)*np.array(likelihoods)
    idx_best_post = np.argmax(posteriors)

    for i in range(num_candidates):
        best = '<----' if i==idx_best_post else ''
        print('\t'.join(CX[i]), '\tlikelihood = {}\tprior = {}\tposterior = {} {}'
            .format(likelihoods[i], priors[i], posteriors[i],best))

    print('\nThe original sentence was:')
    print('\t' + ' '.join(X))
    print('The right sentence is:')
    print('\t' + ' '.join(CX[idx_best_post]))


    for _ in range(len(alphas)):    
        print('\n\n','-'*50, '\n\n')


Se han truncado las últimas 5000 líneas del flujo de salida.

W = CX[i] = ['an', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'an', 'wish', 'you', 'where', 'here', '</s>']
P(an | <s>, <s>) = 0.0003352183183461574
P(wish | <s>, an) = 9.937888198757765e-05
P(you | an, wish) = 0.04477611940298508
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 3.509426730343075e-17

W = CX[i] = ['as', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'as', 'wish', 'you', 'where', 'here', '</s>']
P(as | <s>, <s>) = 0.00025799793601651185
P(wish | <s>, as) = 9.937888198757765e-05
P(you | as, wish) = 0.04477611940298508
P(where | wish, you) = 0.0002385211687537269
P(here | you, where) = 9.863724853990915e-05
P(W) = 2.7010005225750152e-17

W = CX[i] = ['at', 'wish', 'you', 'where', 'here']
['<s>', '<s>', 'at', 'wish', 'you', 'where', 'here', '</s>']
P(at | <s>, <s>) = 0.0003869969040247678
P(wish | <s>, at) = 9.937888198757765e-05
P(you | at, wish) = 0.044776119

###4. make a long (>20) list of sentences each with 1 or 2 mispelled words to assess the speller : does it really work ?

In [ ]:
sentences = [
    "The qucik brown fox jumps over the lazy dog.",
    "She gaev her all to the project.",
    "The sun also raser.",
    "He was born in Decembr.",
    "I would like two scoops of ice creem please.",
    "The boook on the shelf is red.",
    "The train arived at the staion late.",
    "I love going to the beech in summer.",
    "They're going to the movies tonigth.",
    "I can't believe it's already Aprill.",
    "I enjoy reding mystery novels.",
    "She has a pet cat named Whiskerrs.",
    "My favorit color is bleu.",
    "He plays the guita very well.",
    "The cake smeels delicious.",
    "I'm excited to see the balet tonigt.",
    "This sandwich is really tastey.",
    "I'm going to take a shower before bedd.",
    "The golfer hit the ball into the water hazad.",
    "I need to buy more tolitepaper."
]

for test_sentence in sentences:
    print(f"Original sentence: {test_sentence}")
    test_sentence = test_sentence[:-1].lower()  # Remove the period at the end and convert to lowercase
    X = test_sentence.split(' ')

    # Check if all words belong to the vocabulary
    for x in X:
        if x not in vocabulary:
            print(f"{x} doesn't belong to the vocabulary, skipping the sentence.")
            continue

    CX = [X]  # no errors
    for i in range(len(X)):  # one misspelled word at a time
        if X[i] in vocabulary:
            for cw in close_words[X[i]]:
                if cw != X[i]:
                    C = X.copy()
                    C[i] = cw
                    CX.append(C)
        else:
            pass  # let it be as it is

    likelihoods = []
    for W in CX:
        PXW = 1.0
        for x, w in zip(X, W):
            if w == x:
                pxw = alpha
            else:
                close_to_x = close_words[x]  # includes x itself
                pxw = (1 - alpha) / (len(close_to_x) - 1)  # so we subtract 1
            PXW *= pxw
        likelihoods.append(PXW)

    likelihoods = np.array(likelihoods)
    idx_most_likely = likelihoods.argmax()

    priors = []
    for W in CX:
        priors.append(P(W))

    posteriors = np.array(priors) * np.array(likelihoods)
    idx_best_post = np.argmax(posteriors)

    print(f"The corrected sentence is: {' '.join(CX[idx_best_post])}.")
    print()



Original sentence: The qucik brown fox jumps over the lazy dog.
qucik doesn't belong to the vocabulary, skipping the sentence.
brown doesn't belong to the vocabulary, skipping the sentence.
fox doesn't belong to the vocabulary, skipping the sentence.
jumps doesn't belong to the vocabulary, skipping the sentence.
lazy doesn't belong to the vocabulary, skipping the sentence.
dog doesn't belong to the vocabulary, skipping the sentence.
The corrected sentence is: the qucik brown fox jumps over the lazy dog.

Original sentence: She gaev her all to the project.
gaev doesn't belong to the vocabulary, skipping the sentence.
project doesn't belong to the vocabulary, skipping the sentence.
The corrected sentence is: she gaev her all to the project.

Original sentence: The sun also raser.
sun doesn't belong to the vocabulary, skipping the sentence.
also doesn't belong to the vocabulary, skipping the sentence.
raser doesn't belong to the vocabulary, skipping the sentence.
The corrected sentence is

NO IT DOES NOT WORK AS WORDS AREN´T IN THE VOCABULARY AS IT IS AN OLD BOOK

### 5. same for a second list of sentences withouth any spelling error (but different from any one in the selected part of the book), are there any errors ?

In [ ]:
correct_sentences = [
    "The weather today is nice and sunny.",
    "She enjoys reading books in her free time.",
    "He went for a long walk in the park.",
    "The cake needs more sugar to taste better.",
    "I prefer tea over coffee in the morning.",
    "Please hand me the blue pen on the table.",
    "The children played happily in the playground.",
    "It is important to get enough sleep every night.",
    "The movie we watched last night was interesting.",
    "She is looking forward to her vacation next month."
]

for test_sentence in correct_sentences:
    print(f"Original sentence: {test_sentence}")
    test_sentence = test_sentence[:-1].lower()  # Remove the period at the end and convert to lowercase
    X = test_sentence.split(' ')

    # Check if all words belong to the vocabulary
    for x in X:
        if x not in vocabulary:
            print(f"{x} doesn't belong to the vocabulary, skipping the sentence.")
            continue

    CX = [X]  # no errors
    for i in range(len(X)):  # one misspelled word at a time
        if X[i] in vocabulary:
            for cw in close_words[X[i]]:
                if cw != X[i]:
                    C = X.copy()
                    C[i] = cw
                    CX.append(C)
        else:
            pass  # let it be as it is

    likelihoods = []
    for W in CX:
        PXW = 1.0
        for x, w in zip(X, W):
            if w == x:
                pxw = alpha
            else:
                close_to_x = close_words[x]  # includes x itself
                pxw = (1 - alpha) / (len(close_to_x) - 1)  # so we subtract 1
            PXW *= pxw
        likelihoods.append(PXW)

    likelihoods = np.array(likelihoods)
    idx_most_likely = likelihoods.argmax()

    priors = []
    for W in CX:
        priors.append(P(W))

    posteriors = np.array(priors) * np.array(likelihoods)
    idx_best_post = np.argmax(posteriors)

    print(f"The corrected sentence is: {' '.join(CX[idx_best_post])}.")
    print()

Original sentence: The weather today is nice and sunny.
weather doesn't belong to the vocabulary, skipping the sentence.
today doesn't belong to the vocabulary, skipping the sentence.
nice doesn't belong to the vocabulary, skipping the sentence.
sunny doesn't belong to the vocabulary, skipping the sentence.
The corrected sentence is: the weather today is nice and sunny.

Original sentence: She enjoys reading books in her free time.
enjoys doesn't belong to the vocabulary, skipping the sentence.
reading doesn't belong to the vocabulary, skipping the sentence.
The corrected sentence is: she enjoys reading books in her free time.

Original sentence: He went for a long walk in the park.
park doesn't belong to the vocabulary, skipping the sentence.
The corrected sentence is: I went for a long walk in the park.

Original sentence: The cake needs more sugar to taste better.
needs doesn't belong to the vocabulary, skipping the sentence.
sugar doesn't belong to the vocabulary, skipping the sent

### 6. extra points: what if you change the tri-gram language model by the neural network of exercise 2 ? 

An error that I can´t understand occurs with CUDA

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)



class FixedWindow(Dataset):
    def __init__(self, words, length_window):
        super().__init__()
        self.length_window = length_window
        # TODO:
        # compute the vocabulary = list of unique words in 'words',
        self.vocabulary = list(set([word for word in words]))
        
        # then assign a unique id number to each word in the vocabulary, 
        self.id_vocabulary = {i: word for i, word in enumerate(self.vocabulary)}
        
        # and finally compute a list of ids, one per word in 'words'
        
        # Create a reverse mapping to convert words to their ids, set the word as key and the id as value
        self.word_to_id = {word: i for i, word in self.id_vocabulary.items()}

        # Compute a list of ids, one per word in 'words'
        self.word_ids = [self.word_to_id[word] for word in words]


    def __len__(self):
        return len(self.word_ids) - self.length_window

    def __getitem__(self, idx):
        #TODO:
        '''
        returns a pair of tensors (first_ids, last_id) where
        first_ids are the ids of the words starting at index
        idx with length length_window-1, and last_id is the
        id at position idx+self.length_window-1, next to first_ids
        '''
        # Get the first_ids list of length length_window - 1
        first_ids_list = self.word_ids[idx: idx + self.length_window - 1]

        # Get the last_id at position idx + self.length_window - 1
        last_id_list = self.word_ids[idx + self.length_window - 1]

        # Convert the lists into tensors using torch.as_tensor()
        first_ids = torch.as_tensor(first_ids_list, dtype=torch.long)
        last_id = torch.as_tensor(last_id_list, dtype=torch.long)

        return first_ids, last_id


cuda:0


In [ ]:
class SimpleNNLM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(SimpleNNLM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.hidden = nn.Linear(embed_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embeds = self.embedding(x).sum(dim=1)
        hidden_out = torch.relu(self.hidden(embeds))
        out = self.output(hidden_out)
        return out


In [ ]:

# Load the dataset and create DataLoader
length_window = 5
dataset = FixedWindow(words, length_window)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create and train the neural network model
vocab_size = len(dataset.vocabulary)
embed_dim = 128
hidden_dim = 128
output_dim = vocab_size

model = SimpleNNLM(vocab_size, embed_dim, hidden_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, target = batch
        inputs, target = inputs.to(device), target.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')


def P_NNLM(W):
    input_ids = [dataset.word_to_id.get(word, -1) for word in W]
    valid_input_ids = [i for i in input_ids if i != -1]
    X = torch.tensor(valid_input_ids).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model(X)
    probs = torch.nn.functional.softmax(pred, dim=1)
    prob_sequence = 1.0
    for i, w in enumerate(W[1:]):
        if input_ids[i] == -1:  # If the word is not in the vocabulary, assign a low probability
            prob_sequence *= 1e-6
        else:
            prob_sequence *= probs[0][dataset.word_to_id[w]].item()
    return prob_sequence



RuntimeError: ignored

In [ ]:
import numpy as np
import random
import string

def generate_misspelled_word(word):
    letters = string.ascii_letters
    operation = random.choice(["replace", "insert", "delete"])

    if operation == "replace":
        index = random.randint(0, len(word) - 1)
        new_char = random.choice(letters)
        misspelled_word = word[:index] + new_char + word[index + 1:]
    elif operation == "insert":
        index = random.randint(0, len(word))
        new_char = random.choice(letters)
        misspelled_word = word[:index] + new_char + word[index:]
    else:
        index = random.randint(0, len(word) - 1)
        misspelled_word = word[:index] + word[index + 1:]

    return misspelled_word

test_sentences = [
    "The weather today is nice and sunny.",
    "She was tired but continued working.",
    "I love eating chocolate ice cream.",
    "Please take out the trash before leaving.",
    "The new movie was absolutely fantastic.",
    "He always forgets his keys at home.",
    "Let's go to the park this weekend.",
    "The dog chased the squirrel up the tree.",
    "I can't believe how fast time flies.",
    "Make sure to lock the door when you leave.",
]

for sentence in test_sentences:
    words = sentence.split()

    misspelled_indices = random.sample(range(len(words)), random.randint(1, 2))
    misspelled_words = [
        generate_misspelled_word(words[i]) if i in misspelled_indices else words[i]
        for i in range(len(words))
    ]

    misspelled_sentence = " ".join(misspelled_words)
    print("Original sentence:", sentence)
    print("Misspelled sentence:", misspelled_sentence)
    print("Probability:", P_NNLM(misspelled_words))
    print()
